# libs & config

In [22]:
# ajustando o caminho que o python vai procurar os arquivos
import os
import pandas as pd
import sys

# pega o diretorio acima de noteboosks, ou seja o do projeto principal
# current_dir = os.path.dirname(os.path.abspath(__file__)) - # isso nao funciona no jupyter , pois o __file__ e exclusivo de scrpit py
current_dir = os.getcwd()

# muda o diretorio para o projeto --
# isso é necessario para que o python consiga importar os modulos do projeto
# caso contrario, ele nao consegue achar os modulos
project_root = os.path.join(current_dir, '..')
sys.path.append(project_root)

# PIPE

## 1 - Coleta e Identificação da Origem dos Dados (extract)

### 1.1 Recebimento do arquivo de origem & 1.2 Classificação da Planilha (Tipo e Finalidade)

In [38]:
from Scripts.Extract import ReadArchive, ConvertXml_toExcel, GetFile
# checa a apsta e se achar um arquivo, pega o path extençaõ para usar, SE NAO TIVER AS EXTENSAO ACEITA RETORNA ERRO
# ja esta fazendo o 2.3

found_excel_path = GetFile('xlsx')

if found_excel_path:  # vai direto
    file_path = found_excel_path
    df_prod = ReadArchive(file_path, 1)
else:
    found_excel_path = GetFile('xml')
    if found_excel_path:  # converte e depois le
        file_path = ConvertXml_toExcel(found_excel_path)
        df_prod = ReadArchive(file_path, 0)




Arquivo encontrado: d:\kaue\projetos\pipeline_estoque_bling_masquerade\Data\ESTOQUE MASQUERADE.XLSX


## 2. Classificação da Planilha (Tipo e Finalidade)


### 2.1 Tipo de produto & 2.2 Tipo de movimentação

## 4. 4. Transformação e Validação (Transform)
### 4.1 Carregar produtos já existentes no Bling via API

In [ ]:
from Scripts.Extract import GetProdutos, GetToken

token = GetToken() # precisa estar no arquivo .env
df_prod_bling = GetProdutos(token) # ja devolve em um DataFrame



In [39]:
df_prod

# --  função PadronizeDataFrame
# 1. limpe os que nao fomos usar vazio e criar a coluna grade
# 2. transforme as colunas de tamnho em chaves de um diconario, apenas coluans que tiverem valor, adione na coluna grade
# 3. adicone os valores no diconario com base da scolunas que esta no  diconario, apois adicona na grade apaga as outras colunas
# 4. crie um produto(deve seguir o padro bling "cod,cor[setiver],tam") para cada chave da grade,e crie a coluna quant aond eela receb o respectivo valor da chave
# 5. veja quais produtos nao bate com o da bling, e crie uma coluna de status, se esta ok(true) ou nao(false)
# 6. compare com os produtos da bling, concertando codigos errados atravez de quanto for parecido o nome e cod se concertar atribui ok
# 7. compare novamente , se haver ainda dierentes cria um novo df para criar os produtos que nao existem na bling

# def PadronizeDataFrame(df_prod, df_prod_bling):
# 1. limpe os que nao fomos usar vazio e criar a coluna grade
df_prod.rename(columns={'total:': 'total'}, inplace=True)

tamanhos = ['U', 'PP', 'P', 'M', 'G', 'GG']

df_prod[tamanhos] = df_prod[tamanhos].astype(float)  # converte as colunas de tamanho para float, caso nao seja
df_prod["total"] = df_prod[tamanhos].sum(axis=1) # nao ta pegando da forula do excel
df_prod = df_prod.query('total != 0') # faz o papel do dropna
df_prod.reset_index(inplace=True)
df_prod['grade'] = ''

# 2. transforme as colunas de tamnho em chaves de um diconario, apenas coluans que tiverem valor, adione na coluna grade
grades = []
for x in df_prod.index:
    dict_grade = {}
    for tamanho in tamanhos:
        if not pd.isna(df_prod.loc[x, tamanho]): 
            dict_grade[tamanho] = df_prod.loc[x, tamanho]
    grades.append(dict_grade)

# 3. adicone os valores no diconario com base da scolunas que esta no  diconario, apois adicona na grade apaga as outras colunas
df_prod['grade'] = grades
df_prod.drop(columns=tamanhos, inplace=True)





In [40]:
df_prod

,index,CÓDIGO,INFANTIL MENINOS,estoq. Min,total,grade
0,2,HW017,Capa Infantil Preta Bember 3/6 anos P,20,15.0,{'P': 15.0}
1,3,HW 018,Capa Infantil Preta Bember 8/12 anos M,20,2.0,{'M': 2.0}
2,4,HW 115,Capa Cetim c/ Gola (Preta / Vermelha),20,118.0,"{'P': 15.0, 'M': 17.0, 'G': 86.0}"
3,5,HW 116,Capa Draculinha Cacharrel c/ Gola,20,86.0,"{'P': 81.0, 'M': 3.0, 'G': 2.0}"
4,6,HW 121,Vâmpiro Dimitri Colete de Amarrar (Capa + Cole...,20,243.0,"{'P': 17.0, 'M': 1.0, 'G': 200.0, 'GG': 25.0}"
5,7,HW 122,El Diablo Infantil (Colete + Capa + Jabô + Tiara),20,58.0,"{'P': 40.0, 'M': 6.0, 'G': 8.0, 'GG': 4.0}"
6,8,HW 124,Assombração,20,81.0,"{'P': 73.0, 'M': 4.0, 'G': 4.0}"
7,9,HW 125,Fantasminha Gaspar,20,32.0,"{'P': 23.0, 'M': 3.0, 'G': 6.0}"
8,11,HW 146,Vampirinho Igor Luxo (Capa + Colete + Jabô ),20,7.0,"{'P': 6.0, 'M': 1.0}"
9,12,HW 161,Kit Caveirinha Mexicana (Colete + Gravata),20,118.0,"{'P': 35.0, 'M': 50.0, 'G': 33.0}"
